In [1]:
# [1] Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from prophet import Prophet
from sklearn.metrics import mean_absolute_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')


In [2]:
# [2] Carregar CSV
def load_csv(path='data/btc_limpo.csv'):
    df = pd.read_csv(path)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date').reset_index(drop=True)
    # garantir coluna Close numérica
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
    df = df.dropna(subset=['Close']).reset_index(drop=True)
    return df

df = load_csv()
print('Dataset:', df['Date'].min(), '->', df['Date'].max(), '| pontos =', len(df))
df.tail()


Dataset: 2020-06-01 00:00:00 -> 2025-05-31 00:00:00 | pontos = 1826


,Date,Close,Hight,Low,Open,Volume
1821,2025-05-27,108994.640625,110744.210938,107609.554688,109440.406250,57450176272
1822,2025-05-28,107802.328125,109298.289062,106812.929688,108992.171875,49155377493
1823,2025-05-29,105641.757812,108910.046875,105374.398438,107795.570312,56022752042
1824,2025-05-30,103998.570312,106308.945312,103685.789062,105646.210938,57655287183
1825,2025-05-31,104638.093750,104927.101562,103136.117188,103994.718750,38997843858


In [3]:
# [3] Split em treino e teste (últimos 60 dias)
def train_test_split(df, test_size=60):
    train = df.iloc[:-test_size].copy()
    test = df.iloc[-test_size:].copy()
    return train, test

train_df, test_df = train_test_split(df)
print('Treino:', train_df.shape, '| Teste:', test_df.shape)


Treino: (1766, 6) | Teste: (60, 6)


In [5]:
# [4] Avaliação
def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def evaluate(true, pred, name):
    print(f'{name} MAPE: {mape(true, pred):.2f}%')


In [6]:
# [5] Prophet
def run_prophet(train_df, test_df, horizon=7):
    prophet_df = train_df[['Date','Close']].rename(columns={'Date':'ds','Close':'y'})
    prophet_df['ds'] = pd.to_datetime(prophet_df['ds'])
    prophet_df['y'] = prophet_df['y'].astype(float)

    model = Prophet()
    model.fit(prophet_df)

    future = model.make_future_dataframe(periods=len(test_df), freq='D')
    forecast = model.predict(future)

    preds = forecast[['ds','yhat']].tail(len(test_df)).reset_index(drop=True)
    preds['yhat'] = preds['yhat'].astype(float)
    return preds['yhat'].values


In [7]:
# [6] LSTM
def run_lstm(train_df, test_df, lookback=30, epochs=10, horizon=7):
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled = scaler.fit_transform(train_df[['Close']])

    X, y = [], []
    for i in range(lookback, len(scaled)):
        X.append(scaled[i-lookback:i, 0])
        y.append(scaled[i, 0])
    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X.shape[1],1)))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X, y, epochs=epochs, batch_size=32, verbose=0)

    # Previsão rolling no teste
    test_scaled = scaler.transform(test_df[['Close']])
    inputs = np.concatenate((scaled[-lookback:], test_scaled))
    
    X_test, y_test = [], []
    for i in range(lookback, len(inputs)):
        X_test.append(inputs[i-lookback:i, 0])
        y_test.append(inputs[i, 0])
    X_test, y_test = np.array(X_test), np.array(y_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    preds_scaled = model.predict(X_test)
    preds = scaler.inverse_transform(preds_scaled)
    return preds.ravel()


In [8]:
# [7] Rodar experimentos
horizons = [1, 3, 7]

for h in horizons:
    # Prophet
    prophet_preds = run_prophet(train_df, test_df, horizon=h)
    evaluate(test_df['Close'].values[:len(prophet_preds)], prophet_preds, f'Prophet {h}d')

    plt.figure(figsize=(10,5))
    plt.plot(test_df['Date'][:len(prophet_preds)], test_df['Close'][:len(prophet_preds)], label='Real')
    plt.plot(test_df['Date'][:len(prophet_preds)], prophet_preds, label=f'Prophet {h}d')
    plt.title(f'Prophet Previsão {h} dias')
    plt.legend()
    plt.savefig(f'prophet_{h}d.png')
    plt.close()

    # LSTM
    lstm_preds = run_lstm(train_df, test_df, horizon=h)
    evaluate(test_df['Close'].values[:len(lstm_preds)], lstm_preds, f'LSTM {h}d')

    plt.figure(figsize=(10,5))
    plt.plot(test_df['Date'][:len(lstm_preds)], test_df['Close'][:len(lstm_preds)], label='Real')
    plt.plot(test_df['Date'][:len(lstm_preds)], lstm_preds, label=f'LSTM {h}d')
    plt.title(f'LSTM Previsão {h} dias')
    plt.legend()
    plt.savefig(f'lstm_{h}d.png')
    plt.close()

print("Gráficos salvos como prophet_*.png e lstm_*.png")


23:24:40 - cmdstanpy - INFO - Chain [1] start processing
23:24:40 - cmdstanpy - INFO - Chain [1] done processing


Prophet 1d MAPE: 13.82%
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 369ms/step
LSTM 1d MAPE: 2.47%


23:24:56 - cmdstanpy - INFO - Chain [1] start processing
23:24:56 - cmdstanpy - INFO - Chain [1] done processing


Prophet 3d MAPE: 13.82%
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 367ms/step
LSTM 3d MAPE: 4.03%


23:25:09 - cmdstanpy - INFO - Chain [1] start processing
23:25:09 - cmdstanpy - INFO - Chain [1] done processing


Prophet 7d MAPE: 13.82%
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 699ms/step
LSTM 7d MAPE: 2.83%
Gráficos salvos como prophet_*.png e lstm_*.png
